In [10]:
import os
import pandas as pd
from tqdm import tqdm

datasets = ["Round1_T2D", "Round3", "2T_2020", "Round4", "HardTablesR2", "HardTablesR3"]


def get_ne_cols_and_correct_qids(table_name, cea_gt):
    correct_qids = {}
    filtered_cea_gt = cea_gt[cea_gt[0] == table_name]
    ne_cols = {str(col_index) for col_index in filtered_cea_gt[2].unique()}
    for _, row in filtered_cea_gt.iterrows():
        qid = row[3].split("/")[-1]
        correct_qids[f"{int(row[1])-1}-{row[2]}"] = qid
    return ne_cols, correct_qids


for dataset in datasets:
    dataset = "HardTablesR3"
    cea_gt = pd.read_csv(f"./Datasets/{dataset}/gt/cea.csv", header=None)
    tables = os.listdir(f"./Datasets/{dataset}/tables")
    for table in tqdm(tables, desc="Processing tables..."):
        if table.endswith(".csv"):
            df = pd.read_csv(f"./Datasets/{dataset}/tables/{table}")
            table_name = table.split(".csv")[0]
            ne_cols, correct_qids = get_ne_cols_and_correct_qids(table_name, cea_gt)
            break    
    break

Processing tables...:   0%|          | 0/7207 [00:00<?, ?it/s]


In [11]:
ne_cols, correct_qids

({'0'},
 {'0-0': 'Q1084490',
  '1-0': 'Q1046685',
  '2-0': 'Q957787',
  '3-0': 'Q882739',
  '4-0': 'Q879904'})

In [ ]:
import os
import pandas as pd
import requests
import json
from tqdm import tqdm

datasets = ["Round1_T2D", "Round3", "2T_2020", "Round4_2020", "HardTablesR2", "HardTablesR3"]

def get_ne_cols_and_correct_qids(table_name, cea_gt):
    correct_qids = {}
    filtered_cea_gt = cea_gt[cea_gt[0] == table_name]
    ne_cols = {str(col_index) for col_index in filtered_cea_gt[2].unique()}
    for _, row in filtered_cea_gt.iterrows():
        qid = row[3].split("/")[-1]
        correct_qids[f"{int(row[1])-1}-{row[2]}"] = qid
    return ne_cols, correct_qids

# Function to call the NER API endpoint
def classify_columns_with_api(df, type="accurate"):
    payload = {
        "json": [df.T.astype(str).values.tolist()]  # Transpose to align columns and convert to list of rows
    }
    url = f'https://lamapi.hel.sintef.cloud/sti/column-analysis?model_type={type}&token=lamapi_demo_2023'
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    
    if response.status_code == 200:
        return response.json()  # Return JSON response with NER classifications
    else:
        print(f"Error with API call: {response.status_code}")
        return None

# Main processing loop
for dataset in datasets:
    cea_gt = pd.read_csv(f"./Datasets/{dataset}/gt/cea.csv", header=None)
    tables = os.listdir(f"./Datasets/{dataset}/tables")
    
    for table in tqdm(tables, desc="Processing tables..."):
        if table.endswith(".csv"):
            df = pd.read_csv(f"./Datasets/{dataset}/tables/{table}")
            table_name = table.split(".csv")[0]

            # Get NE columns and correct QIDs from GT
            ne_cols, correct_qids = get_ne_cols_and_correct_qids(table_name, cea_gt)
            
            # Call the NER API to classify columns
            ner_response = classify_columns_with_api(df)
            if ner_response:
                lit_cols = {}  # Initialize LIT columns dictionary
                for col_idx, ner_info in ner_response.items():
                    print(ner_info)
                    col_num = str(ner_info["index_column"])  # Get column index as string
                    classification = ner_info["classification"]  # Get NER or LIT type
                    tag = ner_info["tag"]

                    # Update NE columns from GT
                    if col_num in ne_cols:
                        # If GT classifies as NE, we trust the GT and update with the NER type
                        if tag == "LIT":
                            classification = "OTHER"
                        ne_cols[col_num] = classification
                    elif tag == "NE":
                        # If API classifies as NE and not in GT, add to ne_cols
                        ne_cols[col_num] = classification
                    elif tag == "LIT":
                        # If API classifies as LIT, add to lit_cols
                        lit_cols[col_num] = classification
        break
    break  # Keep the break for testing purposes

In [7]:
ner_response

[{'table_1': {'0': {'index_column': 0,
    'tag': 'LIT',
    'classification': 'NUMBER',
    'datatype': 'NUMBER',
    'probabilities': {'NUMBER': 1.0}},
   '1': {'index_column': 1,
    'tag': 'NE',
    'classification': 'OTHER',
    'datatype': 'OTHER',
    'probabilities': {'LOCATION': 0.06,
     'PERSON': 0.02,
     'OTHER': 0.84,
     'STRING': 0.06}},
   '2': {'index_column': 2,
    'tag': 'LIT',
    'classification': 'NUMBER',
    'datatype': 'NUMBER',
    'probabilities': {'NUMBER': 1.0, 'DATE': 1.0}},
   '3': {'index_column': 3,
    'tag': 'NE',
    'classification': 'PERSON',
    'datatype': 'PERSON',
    'probabilities': {'PERSON': 1.0}},
   '4': {'index_column': 4,
    'tag': 'LIT',
    'classification': 'NUMBER',
    'datatype': 'NUMBER',
    'probabilities': {'NUMBER': 1.0}}}}]

In [6]:
ne_cols, lit_cols, tag, ner_response

NameError: name 'tag' is not defined

In [55]:
df

,col0,col1,col2
0,Somosierra,Autovía A-1,Spain
1,Fotu La,National Highway 1,India
2,Zojila Pass,National Highway 1,India
3,Jablunkov Pass,European route E75,Czech Republic
4,Wildhaus Pass,Main road 16,Switzerland
5,Pacific Grade Summit,California State Route 4,United States of America
6,Mahoosuc Notch,Appalachian Trail,United States of America
7,Zealand Notch,Appalachian Trail,United States of America
8,Summit Pass,Alaska Highway,Canada
9,Muncho Pass,Alaska Highway,Canada


In [ ]:
import os
import pandas as pd
import requests
import json
from pymongo import MongoClient, ASCENDING
from tqdm import tqdm

# MongoDB connection
client = MongoClient("mongodb://mongodb:27017/")
db = client["crocodile_db"]
input_collection = db["input_data"]
table_trace_collection = db["table_trace"]
dataset_trace_collection = db["dataset_trace"]
process_queue = db["process_queue"]

# Ensure indexes for uniqueness and performance
input_collection.create_index([("dataset_name", ASCENDING), ("table_name", ASCENDING), ("row_id", ASCENDING)], unique=True)
table_trace_collection.create_index([("dataset_name", ASCENDING), ("table_name", ASCENDING)], unique=True)
dataset_trace_collection.create_index([("dataset_name", ASCENDING)], unique=True)
process_queue.create_index([("dataset_name", ASCENDING), ("table_name", ASCENDING)], unique=True)
process_queue.create_index([("status", ASCENDING)])  # Ensure fast retrieval of items by status

datasets = ["Round1_T2D", "Round3", "2T_2020", "Round4_2020", "HardTablesR2", "HardTablesR3"]

# Function to get NE columns and correct QIDs from the GT file
def get_ne_cols_and_correct_qids(table_name, cea_gt):
    correct_qids = {}
    filtered_cea_gt = cea_gt[cea_gt[0] == table_name]
    # Initialize ne_cols as a dict where the value (NER type) is None initially
    ne_cols = {str(col_index): None for col_index in filtered_cea_gt[2].unique()}
    for _, row in filtered_cea_gt.iterrows():
        qid = row[3].split("/")[-1]
        correct_qids[f"{int(row[1])-1}-{row[2]}"] = qid
    return ne_cols, correct_qids

# Function to call the NER API endpoint
def classify_columns_with_api(df, type="accurate"):
    payload = {
        "json": df.T.astype(str).values.tolist()  # Transpose to align columns and convert to list of rows
    }
    url = f'https://lamapi.hel.sintef.cloud/sti/column-analysis?model_type={type}&token=lamapi_demo_2023'
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    
    if response.status_code == 200:
        return response.json()[0]["table_1"]  # Return JSON response with NER classifications
    else:
        print(f"Error with API call: {response.status_code}")
        return None

# Function to onboard data into MongoDB
def onboard_data(dataset_name, table_name, df, ne_cols, correct_qids, ner_response):
    lit_cols = {}
    all_columns = set([str(i) for i in range(len(df.columns))])  # Set of all columns (as strings)

    # Parse the API response for NE and LIT columns
    if ner_response:
        for col_idx, ner_info in ner_response.items():
            col_num = str(ner_info["index_column"])
            classification = ner_info["classification"]
            tag = ner_info["tag"]
            
            if col_num in ne_cols:
                # If GT classifies as NE, we trust the GT and update with NER type
                if tag == "LIT":
                    classification = "OTHER"
                ne_cols[col_num] = classification
            elif tag == "LIT":
                # If API classifies as LIT, add to LIT columns
                lit_cols[col_num] = classification

    # Compute unclassified columns by subtracting NE and LIT columns from all columns
    classified_ne_columns = set(ne_cols.keys())
    classified_lit_columns = set(lit_cols.keys())
    classified_columns = classified_ne_columns.union(classified_lit_columns)
    unclassified_columns = all_columns.difference(classified_columns)
    
    # Insert data row by row
    for index, row in df.iterrows():
        document = {
            "dataset_name": dataset_name,
            "table_name": table_name,
            "row_id": index,
            "data": row.to_dict(),
            "classified_columns": {
                "NE": ne_cols,  # Use updated NE columns
                "LIT": lit_cols,  # Use LIT columns from the API response
                "UNCLASSIFIED": list(unclassified_columns)  # Unclassified columns
            },
            "context_columns": [str(i) for i in range(len(df.columns))],  # Context columns
            "correct_qids": correct_qids,  # Correct QIDs from GT
            "status": "TODO"
        }

        # Insert or update the document in MongoDB
        input_collection.update_one(
            {"dataset_name": dataset_name, "table_name": table_name, "row_id": index},
            {"$set": document},
            upsert=True
        )

    # Log onboarding completion for table-level trace
    table_trace_collection.update_one(
        {"dataset_name": dataset_name, "table_name": table_name},
        {"$set": {
            "total_rows": len(df),
            "processed_rows": 0,
            "status": "PENDING"
        }},
        upsert=True
    )

    # Add table to process queue
    process_queue.update_one(
        {"dataset_name": dataset_name, "table_name": table_name},
        {"$set": {
            "status": "QUEUED",
            "table_name": table_name,
            "dataset_name": dataset_name
        }},
        upsert=True
    )

# Main processing loop for onboarding datasets
for dataset in datasets:
    cea_gt = pd.read_csv(f"./Datasets/{dataset}/gt/cea.csv", header=None)
    tables = os.listdir(f"./Datasets/{dataset}/tables")
    
    for table in tqdm(tables, desc=f"Processing tables for dataset {dataset}..."):
        if table.endswith(".csv"):
            df = pd.read_csv(f"./Datasets/{dataset}/tables/{table}")
            table_name = table.split(".csv")[0]

            # Get NE columns and correct QIDs from GT
            ne_cols, correct_qids = get_ne_cols_and_correct_qids(table_name, cea_gt)
            
            # Call the NER API to classify columns
            ner_response = classify_columns_with_api(df, type="fast")

            # Onboard the data into MongoDB
            onboard_data(dataset, table_name, df, ne_cols, correct_qids, ner_response)

    # Initialize dataset-level trace after onboarding all tables in the dataset
    dataset_trace_collection.update_one(
        {"dataset_name": dataset},
        {"$setOnInsert": {
            "total_tables": len(tables),
            "processed_tables": 0,
            "total_rows": 0,  # Updated as tables are processed
            "processed_rows": 0,
            "status": "PENDING"
        }},
        upsert=True
    )
    break

In [16]:
import os
import pandas as pd
import requests
import json
from tqdm import tqdm

datasets = ["Round1_T2D", "Round3", "2T_2020", "Round4_2020", "HardTablesR2", "HardTablesR3"]

def get_ne_cols_and_correct_qids(table_name, cea_gt):
    correct_qids = {}
    filtered_cea_gt = cea_gt[cea_gt[0] == table_name]
    ne_cols = {str(col_index): "NE" for col_index in filtered_cea_gt[2].unique()}
    for _, row in filtered_cea_gt.iterrows():
        qid = row[3].split("/")[-1]
        correct_qids[f"{int(row[1])-1}-{row[2]}"] = qid
    return ne_cols, correct_qids

# Function to call the NER API endpoint with multiple tables
def classify_columns_with_api(tables_data, type="fast"):
    payload = {
        "json": [df.T.astype(str).values.tolist() for df in tables_data]  # Multiple tables as input
    }
    url = f'https://lamapi.hel.sintef.cloud/sti/column-analysis?model_type={type}&token=lamapi_demo_2023'
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    
    if response.status_code == 200:
        return response.json()  # Return JSON response with NER classifications
    else:
        print(f"Error with API call: {response.status_code}")
        return None

# Main processing loop
def process_tables(datasets, max_tables_at_once=1):
    for dataset in datasets:
        cea_gt = pd.read_csv(f"./Datasets/{dataset}/gt/cea.csv", header=None)
        tables = os.listdir(f"./Datasets/{dataset}/tables")
        tables_data = []
        processed_count = 0

        for table in tqdm(tables, desc="Processing tables..."):
            if table.endswith(".csv"):
                df = pd.read_csv(f"./Datasets/{dataset}/tables/{table}")
                table_name = table.split(".csv")[0]

                # Get NE columns and correct QIDs from GT
                ne_cols, correct_qids = get_ne_cols_and_correct_qids(table_name, cea_gt)
                
                # Add table data to list
                tables_data.append(df)
                processed_count += 1

                # If we reach the maximum number of tables, make the API call
                if processed_count >= max_tables_at_once:
                    # Call the NER API to classify columns for these tables
                    ner_response = classify_columns_with_api(tables_data)
                    
                    if ner_response:
                        for table_response in ner_response:
                            table_key = list(table_response.keys())[0]
                            lit_cols = {}  # Initialize LIT columns dictionary
                            for col_idx, ner_info in table_response[table_key].items():
                                col_num = str(ner_info["index_column"])  # Get column index as string
                                classification = ner_info["classification"]  # Get NER or LIT type
                                tag = ner_info["tag"]

                                # Update NE columns from GT
                                if col_num in ne_cols:
                                    # If GT classifies as NE, we trust the GT and update with the NER type
                                    if tag == "LIT":
                                        classification = "OTHER"
                                    ne_cols[col_num] = classification
                                elif tag == "LIT":
                                    # If API classifies as LIT, add to lit_cols
                                    lit_cols[col_num] = classification

                    # Reset tables data for the next batch
                    tables_data = []
                    processed_count = 0
            
        # In case there are remaining tables that haven't been processed yet
        if tables_data:
            ner_response = classify_columns_with_api(tables_data)
        return ne_cols

# Example of running the function
process_tables(datasets, max_tables_at_once=10)  # Adjust the number of tables you want to submit at once

Processing tables...: 100%|██████████| 64/64 [00:07<00:00,  8.34it/s]


{'0': 'NE'}

In [15]:
ne_cols

{'1'}

In [9]:
import os
import pandas as pd
import requests
import json
from tqdm import tqdm

datasets = ["Round1_T2D", "Round3", "2T_2020", "Round4_2020", "HardTablesR2", "HardTablesR3"]

# Function to get NE columns and correct QIDs from the GT file
def get_ne_cols_and_correct_qids(table_name, cea_gt):
    correct_qids = {}
    filtered_cea_gt = cea_gt[cea_gt[0] == table_name]
    # Initialize ne_cols as a dict where the value (NER type) is None initially
    ne_cols = {str(col_index): None for col_index in filtered_cea_gt[2].unique()}
    for _, row in filtered_cea_gt.iterrows():
        qid = row[3].split("/")[-1]
        correct_qids[f"{row[1]}-{row[2]}"] = qid
    return ne_cols, correct_qids

# Function to call the NER API endpoint with multiple tables
def classify_columns_with_api(tables_data, type="fast"):
    payload = {
        "json": [df.T.astype(str).values.tolist() for df in tables_data]  # Multiple tables as input
    }
    url = f'https://lamapi.hel.sintef.cloud/sti/column-analysis?model_type={type}&token=lamapi_demo_2023'
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    
    if response.status_code == 200:
        return response.json()  # Return JSON response with NER classifications
    else:
        print(f"Error with API call: {response.status_code}")
        return None

# Main processing loop
def process_tables(datasets, max_tables_at_once=1):
    for dataset in datasets:
        cea_gt = pd.read_csv(f"./Datasets/{dataset}/gt/cea.csv", header=None)
        tables = os.listdir(f"./Datasets/{dataset}/tables")
        tables_data = []
        processed_count = 0

        for table in tqdm(tables, desc="Processing tables..."):
            if table.endswith(".csv"):
                df = pd.read_csv(f"./Datasets/{dataset}/tables/{table}")
                table_name = table.split(".csv")[0]

                # Get NE columns and correct QIDs from GT
                ne_cols, correct_qids = get_ne_cols_and_correct_qids(table_name, cea_gt)
                
                # Add table data to list
                tables_data.append(df)
                processed_count += 1

                # If we reach the maximum number of tables, make the API call
                if processed_count >= max_tables_at_once:
                    # Call the NER API to classify columns for these tables
                    ner_response = classify_columns_with_api(tables_data)
                    
                    if ner_response:
                        for table_response in ner_response:
                            table_key = list(table_response.keys())[0]
                            lit_cols = {}  # Initialize LIT columns dictionary
                            for col_idx, ner_info in table_response[table_key].items():
                                col_num = str(ner_info["index_column"])  # Get column index as string
                                classification = ner_info["classification"]  # Get NER or LIT type
                                tag = ner_info["tag"]

                                # Update NE columns from GT
                                if col_num in ne_cols:
                                    # If GT classifies as NE, we trust the GT and update with the NER type
                                    if tag == "LIT":
                                        classification = "OTHER"
                                    ne_cols[col_num] = classification
                                elif tag == "NE":
                                    # If API classifies as NE and not in GT, add to ne_cols
                                    ne_cols[col_num] = classification
                                elif tag == "LIT":
                                    # If API classifies as LIT, add to lit_cols
                                    lit_cols[col_num] = classification

                    # Reset tables data for the next batch
                    tables_data = []
                    processed_count = 0
        
        # In case there are remaining tables that haven't been processed yet
        if tables_data:
            ner_response = classify_columns_with_api(tables_data)
        return

# Example of running the function
process_tables(datasets, max_tables_at_once=10)  # Adjust the number of tables you want to submit at once

Processing tables...: 100%|██████████| 64/64 [00:07<00:00,  8.17it/s]


In [ ]:
import os
import pandas as pd
import requests
import json
from pymongo import MongoClient, ASCENDING
from tqdm import tqdm

# MongoDB connection
client = MongoClient("mongodb://mongodb:27017/")
db = client["crocodile_db"]
input_collection = db["input_data"]
table_trace_collection = db["table_trace"]
dataset_trace_collection = db["dataset_trace"]
process_queue = db["process_queue"]

# Ensure indexes for uniqueness and performance
input_collection.create_index([("dataset_name", ASCENDING), ("table_name", ASCENDING), ("row_id", ASCENDING)], unique=True)
table_trace_collection.create_index([("dataset_name", ASCENDING), ("table_name", ASCENDING)], unique=True)
dataset_trace_collection.create_index([("dataset_name", ASCENDING)], unique=True)
process_queue.create_index([("dataset_name", ASCENDING), ("table_name", ASCENDING)], unique=True)
process_queue.create_index([("status", ASCENDING)])  # Ensure fast retrieval of items by status

datasets = ["Round1_T2D", "Round3_2019", "2T_2020", "Round4_2020", "HardTablesR2", "HardTablesR3"]

# Function to get NE columns and correct QIDs from the GT file
def get_ne_cols_and_correct_qids(table_name, cea_gt):
    correct_qids = {}
    filtered_cea_gt = cea_gt[cea_gt[0] == table_name]
    # Initialize ne_cols as a dict where the value (NER type) is None initially
    ne_cols = {str(col_index): None for col_index in filtered_cea_gt[2].unique()}
    for _, row in filtered_cea_gt.iterrows():
        qid = row[3].split("/")[-1]
        correct_qids[f"{int(row[1])-1}-{row[2]}"] = qid
    return ne_cols, correct_qids

# Function to call the NER API with multiple tables at once and random sample of 100 rows
def classify_columns_with_api(tables_data, type="fast", sample_size=100):
    # For each table, take a random sample of 100 rows if the table has more than 100 rows
    sampled_tables = [df.sample(n=min(sample_size, len(df)), random_state=42) for df in tables_data]

    payload = {
        "json": [df.T.astype(str).values.tolist() for df in sampled_tables]  # Multiple tables as input
    }
    url = f'https://lamapi.hel.sintef.cloud/sti/column-analysis?model_type={type}&token=lamapi_demo_2023'
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }

    # Increase the timeout to accommodate longer processing times
    response = requests.post(url, headers=headers, data=json.dumps(payload), timeout=300)
    
    if response.status_code == 200:
        return response.json()  # Return JSON response with NER classifications
    else:
        print(f"Error with API call: {response.status_code}")
        return None

# Function to onboard data into MongoDB
def onboard_data(dataset_name, table_name, df, ne_cols, correct_qids, ner_response):
    lit_cols = {}
    all_columns = set([str(i) for i in range(len(df.columns))])  # Set of all columns (as strings)
    
    # Parse the API response for NE and LIT columns
    if ner_response:
        for col_idx, ner_info in ner_response.items():
            col_num = str(ner_info["index_column"])
            classification = ner_info["classification"]
            tag = ner_info["tag"]
            
            if col_num in ne_cols:
                # If GT classifies as NE, we trust the GT and update with NER type
                if tag == "LIT":
                    classification = "OTHER"
                ne_cols[col_num] = classification
            elif tag == "LIT":
                # If API classifies as LIT, add to LIT columns
                lit_cols[col_num] = classification

    # Compute unclassified columns by subtracting NE and LIT columns from all columns
    classified_ne_columns = set(ne_cols.keys())
    classified_lit_columns = set(lit_cols.keys())
    classified_columns = classified_ne_columns.union(classified_lit_columns)
    unclassified_columns = all_columns.difference(classified_columns)
    
    # Insert data row by row
    for index, row in df.iterrows():
        document = {
            "dataset_name": dataset_name,
            "table_name": table_name,
            "row_id": index,
            "data": row.to_dict(),
            "classified_columns": {
                "NE": ne_cols,  # Use updated NE columns
                "LIT": lit_cols,  # Use LIT columns from the API response
                "UNCLASSIFIED": list(unclassified_columns)  # Unclassified columns
            },
            "context_columns": [str(i) for i in range(len(df.columns))],  # Context columns
            "correct_qids": correct_qids,  # Correct QIDs from GT
            "status": "TODO"
        }

        # Insert or update the document in MongoDB
        input_collection.update_one(
            {"dataset_name": dataset_name, "table_name": table_name, "row_id": index},
            {"$set": document},
            upsert=True
        )

    # Log onboarding completion for table-level trace
    table_trace_collection.update_one(
        {"dataset_name": dataset_name, "table_name": table_name},
        {"$set": {
            "total_rows": len(df),
            "processed_rows": 0,
            "status": "PENDING"
        }},
        upsert=True
    )

    # Add table to process queue
    process_queue.update_one(
        {"dataset_name": dataset_name, "table_name": table_name},
        {"$set": {
            "status": "QUEUED",
            "table_name": table_name,
            "dataset_name": dataset_name
        }},
        upsert=True
    )

# Main processing loop for onboarding datasets
def process_tables(datasets, max_tables_at_once=5, sample_size=100):
    for dataset in datasets:
        cea_gt = pd.read_csv(f"./Datasets/{dataset}/gt/cea.csv", header=None)
        tables = os.listdir(f"./Datasets/{dataset}/tables")
        tables_data = []
        table_names = []
        processed_count = 0

        for table in tqdm(tables, desc=f"Processing tables for dataset {dataset}..."):
            if table.endswith(".csv"):
                df = pd.read_csv(f"./Datasets/{dataset}/tables/{table}")
                table_name = table.split(".csv")[0]

                # Get NE columns and correct QIDs from GT
                ne_cols, correct_qids = get_ne_cols_and_correct_qids(table_name, cea_gt)
                
                # Add table data and name to lists
                tables_data.append(df)
                table_names.append((dataset, table_name, df, ne_cols, correct_qids))
                processed_count += 1

                # If we reach the maximum number of tables, make the API call
                if processed_count >= max_tables_at_once:
                    # Call the NER API to classify columns for these tables (with sampling)
                    ner_responses = classify_columns_with_api(tables_data, sample_size=sample_size)
                    
                    if ner_responses:
                        for table_idx, table_response in enumerate(ner_responses):
                            table_key = list(table_response.keys())[0]  # Extract table key (e.g., 'table_1', 'table_2')
                            dataset, table_name, df, ne_cols, correct_qids = table_names[table_idx]
                            # Onboard the data into MongoDB
                            onboard_data(dataset, table_name, df, ne_cols, correct_qids, table_response[table_key])

                    # Reset tables data for the next batch
                    tables_data = []
                    table_names = []
                    processed_count = 0
            
        # In case there are remaining tables that haven't been processed yet
        if tables_data:
            ner_responses = classify_columns_with_api(tables_data, sample_size=sample_size)
            if ner_responses:
                for table_idx, table_response in enumerate(ner_responses):
                    table_key = list(table_response.keys())[0]  # Extract table key
                    dataset, table_name, df, ne_cols, correct_qids = table_names[table_idx]
                    onboard_data(dataset, table_name, df, ne_cols, correct_qids, table_response[table_key])

        # Initialize dataset-level trace after onboarding all tables in the dataset
        dataset_trace_collection.update_one(
            {"dataset_name": dataset},
            {"$setOnInsert": {
                "total_tables": len(tables),
                "processed_tables": 0,
                "total_rows": 0,  # Updated as tables are processed
                "processed_rows": 0,
                "status": "PENDING"
            }},
            upsert=True
        ) 

# Example of running the function with batching and sampling
process_tables(datasets, max_tables_at_once=10, sample_size=100)

In [ ]:
! pip install column-classifier

In [1]:
import pandas as pd

In [ ]:
import os
import pandas as pd
from column_classifier import ColumnClassifier
from pymongo import MongoClient, ASCENDING
from tqdm import tqdm

# MongoDB connection
client = MongoClient("mongodb://mongodb:27017/")
db = client["crocodile_db"]
input_collection = db["input_data"]
table_trace_collection = db["table_trace"]
dataset_trace_collection = db["dataset_trace"]
process_queue = db["process_queue"]

# Ensure indexes for uniqueness and performance
input_collection.create_index([("dataset_name", ASCENDING), ("table_name", ASCENDING), ("row_id", ASCENDING)], unique=True)
table_trace_collection.create_index([("dataset_name", ASCENDING), ("table_name", ASCENDING)], unique=True)
dataset_trace_collection.create_index([("dataset_name", ASCENDING)], unique=True)
process_queue.create_index([("dataset_name", ASCENDING), ("table_name", ASCENDING)], unique=True)
process_queue.create_index([("status", ASCENDING)])  # Ensure fast retrieval of items by status

datasets = ["Round1_T2D", "Round3_2019", "2T_2020", "Round4_2020", "HardTablesR2", "HardTablesR3"]

# Initialize the column classifier
classifier = ColumnClassifier(model_type='fast')

# Function to get NE columns and correct QIDs from the GT file
def get_ne_cols_and_correct_qids(table_name, cea_gt):
    correct_qids = {}
    filtered_cea_gt = cea_gt[cea_gt[0] == table_name]
    ne_cols = {int(row[3]): None for row in filtered_cea_gt.itertuples()}  # Column index as integer
    for _, row in filtered_cea_gt.iterrows():
        qid = row[3].split("/")[-1]
        correct_qids[f"{int(row[1])-1}-{row[2]}"] = qid
    return ne_cols, correct_qids

# Function to determine tag based on classification
def determine_tag(classification):
    ne_types = ["LOCATION", "ORGANIZATION", "PERSON", "OTHER"]
    if classification in ne_types:
        return "NE"
    return "LIT"

# Function to onboard data into MongoDB
def onboard_data(dataset_name, table_name, df, ne_cols, lit_cols, correct_qids):
    all_columns = {str(i) for i in range(len(df.columns))}

    for index, row in df.iterrows():
        document = {
            "dataset_name": dataset_name,
            "table_name": table_name,
            "row_id": index,
            "data": row.to_dict(),
            "classified_columns": {
                "NE": ne_cols,
                "LIT": lit_cols,
                "UNCLASSIFIED": list(all_columns - (set(ne_cols.keys()) | set(lit_cols.keys())))
            },
            "context_columns": list(all_columns),
            "correct_qids": correct_qids,
            "status": "TODO"
        }

        input_collection.update_one(
            {"dataset_name": dataset_name, "table_name": table_name, "row_id": index},
            {"$set": document},
            upsert=True
        )

    table_trace_collection.update_one(
        {"dataset_name": dataset_name, "table_name": table_name},
        {"$set": {
            "total_rows": len(df),
            "processed_rows": 0,
            "status": "PENDING"
        }},
        upsert=True
    )

    process_queue.update_one(
        {"dataset_name": dataset_name, "table_name": table_name},
        {"$set": {
            "status": "QUEUED",
            "table_name": table_name,
            "dataset_name": dataset_name
        }},
        upsert=True
    )

# Main processing loop for onboarding datasets
def process_tables(datasets, max_tables_at_once=5):
    for dataset in datasets:
        cea_gt = pd.read_csv(f"./Datasets/{dataset}/gt/cea.csv", header=None)
        tables = os.listdir(f"./Datasets/{dataset}/tables")
        tables_data = []
        table_names = []
        processed_count = 0

        for table in tqdm(tables, desc=f"Processing tables for dataset {dataset}..."):
            if table.endswith(".csv"):
                df = pd.read_csv(f"./Datasets/{dataset}/tables/{table}")
                table_name = table.split(".csv")[0]

                df_sampled = df.sample(n=min(100, len(df)), random_state=42)

                ne_cols_gt, correct_qids = get_ne_cols_and_correct_qids(table_name, cea_gt)

                tables_data.append(df_sampled)
                table_names.append((dataset, table_name, df, ne_cols_gt, correct_qids))
                processed_count += 1
                if processed_count >= max_tables_at_once:
                    ner_responses = classifier.classify_multiple_tables(tables_data)
                    if ner_responses:
                        for table_idx, table_response in enumerate(ner_responses):
                            dataset, table_name, df, ne_cols_gt, correct_qids = table_names[table_idx]
                            ne_cols_classified = {}
                            lit_cols_classified = {}
                            for table in table_response:
                                ner_info_columns = table_response[table]
                                for col_name in ner_info_columns:
                                    ner_info = ner_info_columns[col_name]
                                    classification = ner_info["classification"]
                                    tag = determine_tag(classification)
                                
                                    col_idx = df.columns.get_loc(col_name)  # Get the integer index of the column
                                    col_idx_str = str(col_idx)
                        
                                    if col_idx in ne_cols_gt:
                                        # If GT classifies as NE, we trust the GT and update with NER type
                                        if tag == "LIT":
                                            classification = "OTHER"
                                        ne_cols_classified[col_idx_str] = classification
                                    elif tag == "LIT":
                                        lit_cols_classified[col_idx_str] = classification

                            onboard_data(dataset, table_name, df, ne_cols_classified, lit_cols_classified, correct_qids)
                    tables_data = []
                    table_names = []
                    processed_count = 0
            
        if tables_data:
            ner_responses = classifier.classify_multiple_tables(tables_data)
            if ner_responses:
                for table_idx, table_response in enumerate(ner_responses):
                    dataset, table_name, df, ne_cols_gt, correct_qids = table_names[table_idx]
                    ne_cols_classified = {}
                    lit_cols_classified = {}
                    for table_name in table_response:
                        ner_info_columns = table_response[table_name]
                        for col_name in ner_info_columns:
                            ner_info = ner_info_columns[col_name]
                            classification = ner_info["classification"]
                            tag = determine_tag(classification)
    
                            col_idx = df.columns.get_loc(col_name)  # Get the integer index of the column
                            col_idx_str = str(col_idx)
                            
                            if col_idx in ne_cols_gt:
                                # If GT classifies as NE, we trust the GT and update with NER type
                                if tag == "LIT":
                                    classification = "OTHER"
                                ne_cols_classified[col_idx_str] = classification
                            elif tag == "LIT":
                                lit_cols_classified[col_idx_str] = classification
    
                    onboard_data(dataset, table_name, df, ne_cols_classified, lit_cols_classified, correct_qids)

        dataset_trace_collection.update_one(
            {"dataset_name": dataset},
            {"$setOnInsert": {
                "total_tables": len(tables),
                "processed_tables": 0,
                "total_rows": 0,
                "processed_rows": 0,
                "status": "PENDING"
            }},
            upsert=True
        ) 

# Example of running the function with batching
process_tables(datasets, max_tables_at_once=10)

Processing tables for dataset 2T_2020...:  60%|██████    | 108/180 [07:30<07:06,  5.92s/it]

In [30]:
filtered_cea_gt = cea_gt[cea_gt[0] == "58891288_0_1117541047012405958"]
ne_cols = {int(row[3]): None for row in filtered_cea_gt.itertuples()}  
ne_cols

{1: None}

In [27]:
for row in filtered_cea_gt.itertuples():
    print(row)
    break

Pandas(Index=33, _1='58891288_0_1117541047012405958', _2=95, _3=1, _4='https://www.wikidata.org/entity/Q42198')


In [ ]:
cea_gt = pd.read_csv(f"./Datasets/Round1_T2D/gt/cea.csv", header=None)
get_ne_cols_and_correct_qids("58891288_0_1117541047012405958", cea_gt)

In [29]:
row[3]

1

In [14]:
cea_gt

,0,1,2,3
0,50245608_0_871275842592178099,154,0,https://www.wikidata.org/entity/Q194413
1,22864497_0_8632623712684511496,227,0,https://www.wikidata.org/entity/Q219795
2,66009064_0_9148652238372261251,15,0,https://www.wikidata.org/entity/Q558664
3,21362676_0_6854186738074119688,75,1,https://www.wikidata.org/entity/Q309048
4,50245608_0_871275842592178099,186,0,https://www.wikidata.org/entity/Q820753
...,...,...,...,...
8074,58891288_0_1117541047012405958,18,1,https://www.wikidata.org/entity/Q42047
8075,90196673_0_5458330029110291950,382,0,https://www.wikidata.org/entity/Q199203
8076,58891288_0_1117541047012405958,51,1,https://www.wikidata.org/entity/Q135465
8077,53822652_0_5767892317858575530,117,1,https://www.wikidata.org/entity/Q200396


In [17]:
for table in test:
    print(table["table_1"])

{'Rank': {'classification': 'NUMBER', 'probabilities': {'NUMBER': 1.0}}, 'Title': {'classification': 'OTHER', 'probabilities': {'LOCATION': 0.14, 'ORGANIZATION': 0.16, 'PERSON': 0.32, 'OTHER': 0.22, 'STRING': 0.08}}, 'Year': {'classification': 'NUMBER', 'probabilities': {'NUMBER': 1.0, 'DATE': 0.88}}, 'Director(s)': {'classification': 'PERSON', 'probabilities': {'PERSON': 1.0}}, 'Overall Rank': {'classification': 'NUMBER', 'probabilities': {'NUMBER': 1.0}}}


KeyError: 'table_1'

In [1]:
import os
import pandas as pd
from column_classifier import ColumnClassifier
from pymongo import MongoClient, ASCENDING
from tqdm import tqdm

# MongoDB connection
client = MongoClient("mongodb://mongodb:27017/")
db = client["crocodile_db"]
input_collection = db["input_data"]
table_trace_collection = db["table_trace"]
dataset_trace_collection = db["dataset_trace"]
process_queue = db["process_queue"]

# Ensure indexes for uniqueness and performance
def ensure_indexes():
    input_collection.create_index([("dataset_name", ASCENDING), ("table_name", ASCENDING), ("row_id", ASCENDING)], unique=True)
    table_trace_collection.create_index([("dataset_name", ASCENDING), ("table_name", ASCENDING)], unique=True)
    dataset_trace_collection.create_index([("dataset_name", ASCENDING)], unique=True)
    process_queue.create_index([("dataset_name", ASCENDING), ("table_name", ASCENDING)], unique=True)
    process_queue.create_index([("status", ASCENDING)])  # Ensure fast retrieval of items by status

ensure_indexes()

datasets = ["Round1_T2D", "Round3_2019", "2T_2020", "Round4_2020", "HardTablesR2", "HardTablesR3"]

# Initialize the column classifier
classifier = ColumnClassifier(model_type='fast')

# Function to get NE columns and correct QIDs from the GT file
def get_ne_cols_and_correct_qids(table_name, cea_gt):
    filtered_cea_gt = cea_gt[cea_gt[0] == table_name]
    ne_cols = {int(row[3]): None for row in filtered_cea_gt.itertuples()}  # Column index as integer
    correct_qids = {f"{int(row[1])-1}-{row[2]}": row[3].split("/")[-1] for _, row in filtered_cea_gt.iterrows()}
    return ne_cols, correct_qids

# Function to determine tag based on classification
def determine_tag(classification):
    return "NE" if classification in ["LOCATION", "ORGANIZATION", "PERSON", "OTHER"] else "LIT"

# Function to batch onboard data into MongoDB
def onboard_data_batch(dataset_name, table_name, df, ne_cols, lit_cols, correct_qids):
    all_columns = set([str(i) for i in range(len(df.columns))])
    classified_ne_columns = set(ne_cols.keys())
    classified_lit_columns = set(lit_cols.keys())
    unclassified_columns = all_columns - (classified_ne_columns | classified_lit_columns)

    documents = []
    for index, row in df.iterrows():
        documents.append({
            "dataset_name": dataset_name,
            "table_name": table_name,
            "row_id": index,
            "data": row.to_dict(),
            "classified_columns": {
                "NE": ne_cols,
                "LIT": lit_cols,
                "UNCLASSIFIED": list(unclassified_columns)
            },
            "context_columns": list(all_columns),
            "correct_qids": correct_qids,
            "status": "TODO"
        })
    
    if documents:
        input_collection.insert_many(documents)  # Batch insert the documents

    # Update trace collections in MongoDB
    table_trace_collection.update_one(
        {"dataset_name": dataset_name, "table_name": table_name},
        {"$set": {
            "total_rows": len(df),
            "processed_rows": 0,
            "status": "PENDING"
        }},
        upsert=True
    )

    process_queue.update_one(
        {"dataset_name": dataset_name, "table_name": table_name},
        {"$set": {
            "status": "QUEUED",
            "table_name": table_name,
            "dataset_name": dataset_name
        }},
        upsert=True
    )

# Main processing loop for onboarding datasets
def process_tables(datasets, max_tables_at_once=5):
    for dataset in datasets:
        cea_gt = pd.read_csv(f"./Datasets/{dataset}/gt/cea.csv", header=None)
        tables = os.listdir(f"./Datasets/{dataset}/tables")
        batch_tables_data = []
        batch_table_names = []

        for table in tqdm(tables, desc=f"Processing tables for dataset {dataset}..."):
            if table.endswith(".csv"):
                df = pd.read_csv(f"./Datasets/{dataset}/tables/{table}")
                table_name = table.split(".csv")[0]

                df_sampled = df.sample(n=min(100, len(df)), random_state=42)

                # Get NE columns and correct QIDs from GT
                ne_cols_gt, correct_qids = get_ne_cols_and_correct_qids(table_name, cea_gt)

                batch_tables_data.append(df_sampled)
                batch_table_names.append((dataset, table_name, df, ne_cols_gt, correct_qids))

                if len(batch_tables_data) >= max_tables_at_once:
                    process_table_batch(batch_tables_data, batch_table_names)
                    batch_tables_data = []
                    batch_table_names = []

        # Process remaining tables in the batch
        if batch_tables_data:
            process_table_batch(batch_tables_data, batch_table_names)

        # Initialize dataset-level trace after processing all tables
        dataset_trace_collection.update_one(
            {"dataset_name": dataset},
            {"$setOnInsert": {
                "total_tables": len(tables),
                "processed_tables": 0,
                "total_rows": 0,
                "processed_rows": 0,
                "status": "PENDING"
            }},
            upsert=True
        )

# Process a batch of tables
def process_table_batch(batch_tables_data, batch_table_names):
    ner_responses = classifier.classify_multiple_tables(batch_tables_data)

    if ner_responses:
        for table_idx, table_response in enumerate(ner_responses):
            dataset, table_name, df, ne_cols_gt, correct_qids = batch_table_names[table_idx]
            ne_cols_classified = {}
            lit_cols_classified = {}

            # Process each table's response
            table_key = list(table_response.keys())[0]
            ner_info_columns = table_response[table_key]

            for col_name, ner_info in ner_info_columns.items():
                classification = ner_info["classification"]
                tag = determine_tag(classification)

                col_idx = df.columns.get_loc(col_name)  # Get the integer index of the column
                col_idx_str = str(col_idx)

                if col_idx in ne_cols_gt:
                    # Trust GT for NE columns but use NER to update
                    if tag == "LIT":
                        classification = "OTHER"
                    ne_cols_classified[col_idx_str] = classification
                elif tag == "LIT":
                    lit_cols_classified[col_idx_str] = classification

            onboard_data_batch(dataset, table_name, df, ne_cols_classified, lit_cols_classified, correct_qids)

# Example of running the function with batching
process_tables(datasets, max_tables_at_once=10)

Processing tables for dataset HardTablesR3...: 100%|██████████| 7207/7207 [00:52<00:00, 138.46it/s]


In [1]:
import os
import pandas as pd
from column_classifier import ColumnClassifier
from pymongo import MongoClient, ASCENDING
from tqdm import tqdm

# MongoDB connection
client = MongoClient("mongodb://mongodb:27017/")
db = client["crocodile_db"]
input_collection = db["input_data"]
table_trace_collection = db["table_trace"]
dataset_trace_collection = db["dataset_trace"]
process_queue = db["process_queue"]

# Ensure indexes for uniqueness and performance
def ensure_indexes():
    input_collection.create_index([("dataset_name", ASCENDING), ("table_name", ASCENDING), ("row_id", ASCENDING)], unique=True)
    table_trace_collection.create_index([("dataset_name", ASCENDING), ("table_name", ASCENDING)], unique=True)
    dataset_trace_collection.create_index([("dataset_name", ASCENDING)], unique=True)
    process_queue.create_index([("dataset_name", ASCENDING), ("table_name", ASCENDING)], unique=True)
    process_queue.create_index([("status", ASCENDING)])  # Ensure fast retrieval of items by status

ensure_indexes()

In [19]:
import os
import pandas as pd
from column_classifier import ColumnClassifier
from pymongo import MongoClient, ASCENDING
from tqdm import tqdm
from tensorflow.keras.models import load_model
import numpy as np

# MongoDB connection
client = MongoClient("mongodb://mongodb:27017/")
db = client["crocodile_db"]
input_collection = db["training_data"]
doc = input_collection.find_one({})
model = load_model("trained_models/neural_ranker.h5")

In [28]:
def extract_features(candidate):
    """Extract features for the ML model from a candidate."""
    return [candidate['features'].get(feature, 0) for feature in candidate["features"]]

candidates = doc["candidates"]['0']
candidate_features = [extract_features(candidate) for candidate in candidates]
# Convert the features list to a NumPy array
candidate_features_array = np.array(candidate_features)

# Use the model to predict on the array
predictions = model.predict(candidate_features_array)
# Use the probability of the positive class as the score
positive_class_scores = predictions[:, 1]  # Assuming index 1 is the positive class

# Assign the scores to candidates
for candidate, score in zip(candidates, positive_class_scores):
    candidate['score'] = round(score, 2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [24]:
predictions

array([[0.17935325, 0.8206468 ],
       [0.8480819 , 0.15191813],
       [0.8018286 , 0.19817129],
       [0.66995764, 0.3300423 ],
       [0.8378532 , 0.16214679],
       [0.8378532 , 0.16214679],
       [0.93741906, 0.06258096],
       [0.8928639 , 0.10713608],
       [0.96457493, 0.03542513],
       [0.97371554, 0.02628445]], dtype=float32)

In [26]:
positive_class_scores

array([0.8206468 , 0.15191813, 0.19817129, 0.3300423 , 0.16214679,
       0.16214679, 0.06258096, 0.10713608, 0.03542513, 0.02628445],
      dtype=float32)

In [29]:
candidates

[{'id': 'Q19355',
  'name': 'La vita è bella',
  'description': '1997 film directed by Roberto Benigni',
  'types': [{'id': 'Q11424', 'name': 'film'}],
  'features': {'ntoken_mention': 4,
   'length_mention': 15,
   'ntoken_entity': 4,
   'length_entity': 15,
   'popularity': 0.09,
   'ed_score': 1.0,
   'desc': 0.0857,
   'descNgram': 0.0,
   'bow_similarity': 0.3,
   'kind': 1,
   'NERtype': 4,
   'column_NERtype': 4},
  'matched_words': ['drama',
   'la',
   'è',
   'benigni',
   'comedy',
   '116',
   'roberto',
   'bella',
   'vita',
   'holocaust'],
  'score': 0.82},
 {'id': 'Q6544851',
  'name': 'La vita è bella',
  'description': 'soundtrack album to the 1997 film',
  'types': [{'id': 'Q482994', 'name': 'album'}],
  'features': {'ntoken_mention': 4,
   'length_mention': 15,
   'ntoken_entity': 4,
   'length_entity': 15,
   'popularity': 0.0,
   'ed_score': 1.0,
   'desc': 0.0278,
   'descNgram': 0.0,
   'bow_similarity': 0.12,
   'kind': 1,
   'NERtype': 4,
   'column_NERtype':

In [27]:
candidates

[{'id': 'Q19355',
  'name': 'La vita è bella',
  'description': '1997 film directed by Roberto Benigni',
  'types': [{'id': 'Q11424', 'name': 'film'}],
  'features': {'ntoken_mention': 4,
   'length_mention': 15,
   'ntoken_entity': 4,
   'length_entity': 15,
   'popularity': 0.09,
   'ed_score': 1.0,
   'desc': 0.0857,
   'descNgram': 0.0,
   'bow_similarity': 0.3,
   'kind': 1,
   'NERtype': 4,
   'column_NERtype': 4},
  'matched_words': ['drama',
   'la',
   'è',
   'benigni',
   'comedy',
   '116',
   'roberto',
   'bella',
   'vita',
   'holocaust'],
  'score': 0.35},
 {'id': 'Q6544851',
  'name': 'La vita è bella',
  'description': 'soundtrack album to the 1997 film',
  'types': [{'id': 'Q482994', 'name': 'album'}],
  'features': {'ntoken_mention': 4,
   'length_mention': 15,
   'ntoken_entity': 4,
   'length_entity': 15,
   'popularity': 0.0,
   'ed_score': 1.0,
   'desc': 0.0278,
   'descNgram': 0.0,
   'bow_similarity': 0.12,
   'kind': 1,
   'NERtype': 4,
   'column_NERtype':